In [ ]:
!pip install tensorflow tensorflow_hub gradio opencv-python matplotlib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 MB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.1/322.1 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.6/12.6 MB 30.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 2.4 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2


In [ ]:
from google.colab import files

uploaded = files.upload()

Saving pic2.jpg to pic2.jpg


In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import matplotlib.pyplot as plt
import cv2  # For edge detection/preprocessing
import gradio as gr

def preprocess_sketch(image_path, is_sketch=True):
    image = plt.imread(image_path)
    image = image.astype(np.float32) / 255.0

    if is_sketch:
        # Edge detection to enhance hand-drawn features
        gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
        edges = cv2.Canny((gray * 255).astype(np.uint8), 50, 150)
        edges = cv2.cvtColor(edges, cv2.COLOR_GRAY2RGB)
        image = np.clip(image * 0.7 + edges * 0.3, 0, 1)  # Blend edges

    image = image[np.newaxis, ...]  # Add batch dimension
    return image

def style_transfer(content, style):
    # Load content (photo) and style (hand-drawn sketch) images
    content_image = preprocess_sketch(content, is_sketch=False)
    style_image = preprocess_sketch(style, is_sketch=True)

    # Resize style image to 256x256 (model-specific requirement)
    style_image = tf.image.resize(style_image, (256, 256))

    # Load TensorFlow Hub model
    hub_module = hub.load('https://tfhub.dev/google/magenta/arbitrary-image-stylization-v1-256/2')

    # Apply style transfer
    outputs = hub_module(tf.constant(content_image), tf.constant(style_image))
    stylized_image = outputs[0][0].numpy()  # Remove batch dim

    # Postprocess for sketch-like output
    def postprocess(image):
        image = np.clip(image * 255, 0, 255).astype(np.uint8)
        # Add noise to simulate hand-drawn texture
        noise = np.random.randn(*image.shape) * 10
        return np.clip(image + noise, 0, 255).astype(np.uint8)

    final_output = postprocess(stylized_image)
    return final_output

# Gradio interface
gr.Interface(
    fn=style_transfer,
    inputs=[gr.Image(label="Content Image"), gr.Image(label="Style Image")],
    outputs=gr.Image(label="Result"),
    examples=[["photo.jpg", "sketch_style.jpg"]],
    title="Sketch2Real: AI-Powered Artistic Style Transfer"
).launch()


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://40af85e570562c2c2b.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import cv2  # For edge detection/preprocessing
import gradio as gr

# Step 1: Preprocess images for better hand-drawn style transfer
def preprocess_sketch(image, is_sketch=True):
    # Ensure the image has 3 channels (RGB)
    if len(image.shape) == 2:  # Grayscale image
        image = cv2.cvtColor(image, cv2.COLOR_GRAY2RGB)
    elif image.shape[2] == 4:  # RGBA image
        image = cv2.cvtColor(image, cv2.COLOR_RGBA2RGB)

    image = image.astype(np.float32) / 255.0  # Normalize to [0, 1]

    if is_sketch:
        # Edge detection to enhance hand-drawn features
        gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
        edges = cv2.Canny((gray * 255).astype(np.uint8), 50, 150)
        edges = cv2.cvtColor(edges, cv2.COLOR_GRAY2RGB)
        image = np.clip(image * 0.7 + edges * 0.3, 0, 1)  # Blend edges

    image = image[np.newaxis, ...]  # Add batch dimension
    return image

# Step 2: Postprocess for sketch-like output
def postprocess(image):
    image = np.clip(image * 255, 0, 255).astype(np.uint8)
    # Add noise to simulate hand-drawn texture
    noise = np.random.randn(*image.shape) * 10
    return np.clip(image + noise, 0, 255).astype(np.uint8)

# Step 3: Style transfer function
def style_transfer(content_image, style_image):
    # Preprocess content and style images
    content_image = preprocess_sketch(content_image, is_sketch=False)
    style_image = preprocess_sketch(style_image, is_sketch=True)

    # Resize style image to 256x256 (model-specific requirement)
    style_image = tf.image.resize(style_image, (256, 256))

    # Load TensorFlow Hub model
    hub_module = hub.load('https://tfhub.dev/google/magenta/arbitrary-image-stylization-v1-256/2')

    # Apply style transfer
    outputs = hub_module(tf.constant(content_image), tf.constant(style_image))
    stylized_image = outputs[0][0].numpy()  # Remove batch dim

    # Postprocess the output
    final_output = postprocess(stylized_image)
    return final_output

# Step 4: Gradio interface
gr.Interface(
    fn=style_transfer,
    inputs=[gr.Image(label="Content Image"), gr.Image(label="Style Image")],
    outputs=gr.Image(label="Result"),
    examples=[["photo.jpg", "sketch_style.jpg"]],
    title="Sketch2Real: AI-Powered Artistic Style Transfer"
).launch()

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://4d7b6244a21e87f3b0.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import cv2
import gradio as gr

# Step 1: Preprocess images for better hand-drawn style transfer
def preprocess_sketch(image, is_sketch=True):
    # Ensure the image has 3 channels (RGB)
    if len(image.shape) == 2:  # Grayscale image
        image = cv2.cvtColor(image, cv2.COLOR_GRAY2RGB)
    elif image.shape[2] == 4:  # RGBA image
        image = cv2.cvtColor(image, cv2.COLOR_RGBA2RGB)

    image = image.astype(np.float32) / 255.0  # Normalize to [0, 1]

    if is_sketch:
        # Edge detection to enhance hand-drawn features
        gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
        edges = cv2.Canny((gray * 255).astype(np.uint8), 50, 150)
        edges = cv2.cvtColor(edges, cv2.COLOR_GRAY2RGB)
        image = np.clip(image * 0.7 + edges * 0.3, 0, 1)  # Blend edges

    image = image[np.newaxis, ...]  # Add batch dimension
    return image

# Step 2: Postprocess for sketch-like output
def postprocess(image):
    image = np.clip(image * 255, 0, 255).astype(np.uint8)
    # Add noise to simulate hand-drawn texture
    noise = np.random.randn(*image.shape) * 10
    return np.clip(image + noise, 0, 255).astype(np.uint8)

# Step 3: Load TensorFlow Hub model
hub_module = hub.load('https://tfhub.dev/google/magenta/arbitrary-image-stylization-v1-256/2')

# Step 4: Style transfer function with style mixing
def style_transfer(content_image, style_image1, style_image2, style_mix_percentage):
    # Preprocess content and style images
    content_image = preprocess_sketch(content_image, is_sketch=False)
    style_image1 = preprocess_sketch(style_image1, is_sketch=True)
    style_image2 = preprocess_sketch(style_image2, is_sketch=True)

    # Resize style images to 256x256 (model-specific requirement)
    style_image1 = tf.image.resize(style_image1, (256, 256))
    style_image2 = tf.image.resize(style_image2, (256, 256))

    # Blend style embeddings based on the mixing percentage
    style_mix_percentage = style_mix_percentage / 100.0  # Convert percentage to [0, 1]
    mixed_style = (style_image1 * style_mix_percentage) + (style_image2 * (1 - style_mix_percentage))

    # Apply style transfer
    outputs = hub_module(tf.constant(content_image), tf.constant(mixed_style))
    stylized_image = outputs[0][0].numpy()  # Remove batch dim

    # Postprocess the output
    final_output = postprocess(stylized_image)
    return final_output

# Step 5: Gradio interface
gr.Interface(
    fn=style_transfer,
    inputs=[
        gr.Image(label="Content Image"),
        gr.Image(label="Style Image 1"),
        gr.Image(label="Style Image 2"),
        gr.Slider(minimum=0, maximum=100,  label="Style Mix Percentage (Style 1 vs Style 2)")
    ],
    outputs=gr.Image(label="Result"),
    examples=[
        ["/content/download (3) (1).jpeg", "/content/photo.jpg", "/content/download (1).jpeg", 50],
        ["/content/pencil-sketch.jpg", "/content/photo.jpg", "/content/download (1).jpeg", 50],
        ["/content/images.png", "/content/photo.jpg", "/content/download (1).jpeg", 50],
        [ "/content/download.png", "/content/download (5).jpeg", "/content/download (6).jpeg"]



    ],
    title="StyleFusion: Mix Artistic Styles with AI",
    description="Upload a sketch, choose two styles, and adjust the mix percentage to create unique art!"
).launch()

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://771c514bf273047038.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
